In [6]:
# -*- coding: utf-8 -*-
import os
import zipfile
import dataiku
import datetime
import pandas as pd
# Recipe input
in_folder = dataiku.Folder("izSREvYl")
in_path = in_folder.get_path()
# Recipe output
out_folder = dataiku.Folder("gK2EWbIL")
out_path = out_folder.get_path()

AttributeError: 'module' object has no attribute 'dku_flow_variables'

In [ ]:
# Stacking all the input files in one dataframe
file_names = os.listdir(in_path)
results = pd.DataFrame()
for file_name in file_names:
    nv = file_name.replace('.zip', '.csv')
    path = os.path.join(in_path, file_name)
    with zipfile.ZipFile(path, 'r') as z:
        for csv_file in z.namelist():
            if csv_file.endswith(".csv"):
                o = pd.read_csv(z.open(csv_file))
                results = pd.concat((results, o))
print results.shape

In [ ]:
# Parsing the timestamp
def get_day(date):
    return datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d')
results['day'] = results['Start date'].map(get_day)

In [ ]:
# Looping thru the days and produce one file per day
for day in results['day'].unique():
    print "Doing %s..." % day
    df = results[results['day'] == day]
    file_name = 'capitalbikeshare_trip_data_{}.csv'.format(day)
    out = os.path.join(out_path, file_name) 
    df.to_csv(out, sep='|', index= None, quotechar='"')